In [1]:
import os
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

In [2]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
DATA_INTERIM_DIR = os.path.join(data_dir, 'interim')
data_raw_dir

'/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW/data/raw'

In [3]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [ ]:
spark = init_spark()
data_weather = spark.read.csv(os.path.join(data_raw_dir, 'en_climate_daily_QC_702S006_2015_P1D.csv'), header=True)
 
data_weather.schema.names

In [ ]:
with open('schema.txt', 'w') as f:
    for col in data_weather.schema.names:
        print(col, file=f)

In [4]:
# data_towing = spark.read.csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=True)
# data_towing = data_towing.withColumn('DATE_ORIGINE', F.concat(data_towing.DATE_ORIGINE.substr(0,10)))
spark = init_spark()
data_towing = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'towing_needed.data'))
data_weather = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_needed.data'))

In [5]:
data_towing.head(2)

[Row(DATE_ORIGINE='2019-01-10', LONGITUDE_ORIGINE='-73.60370215594651', LATITUDE_ORIGINE='45.5590991399067', Distance_km=0.0484305337462778, MOTIF_REMORQUAGE="Constat d'infraction"),
 Row(DATE_ORIGINE='2019-01-10', LONGITUDE_ORIGINE='-73.599733540326', LATITUDE_ORIGINE='45.453884709812606', Distance_km=0.07233637828159714, MOTIF_REMORQUAGE="Constat d'infraction")]

In [7]:
data_weather.head(2)

[Row(Date_Time='2015-01-01', Year='2015', Month='01', Day='01', Mean_Temp='-5.2', Total_Rain=None, Total_Precip='0.3', Total_Snow=None, Spd_of_Max_Gust='54'),
 Row(Date_Time='2015-01-02', Year='2015', Month='01', Day='02', Mean_Temp='-9.7', Total_Rain=None, Total_Precip='0.2', Total_Snow=None, Spd_of_Max_Gust='48')]

In [8]:
joined_data = data_towing.join(data_weather, data_towing.DATE_ORIGINE==data_weather.Date_Time)
joined_data.head(5)

[Row(DATE_ORIGINE='2015-12-31', LONGITUDE_ORIGINE='-73.5872033592732', LATITUDE_ORIGINE='45.524980545696', Distance_km=0.055999886340502104, MOTIF_REMORQUAGE="Constat d'infraction", Date_Time='2015-12-31', Year='2015', Month='12', Day='31', Mean_Temp='-2.7', Total_Rain=None, Total_Precip='0.7', Total_Snow=None, Spd_of_Max_Gust='41'),
 Row(DATE_ORIGINE='2015-12-31', LONGITUDE_ORIGINE='-73.6222559028091', LATITUDE_ORIGINE='45.585753168523894', Distance_km=0.1698879856139788, MOTIF_REMORQUAGE="Constat d'infraction", Date_Time='2015-12-31', Year='2015', Month='12', Day='31', Mean_Temp='-2.7', Total_Rain=None, Total_Precip='0.7', Total_Snow=None, Spd_of_Max_Gust='41'),
 Row(DATE_ORIGINE='2015-12-31', LONGITUDE_ORIGINE='-73.6006920362305', LATITUDE_ORIGINE='45.535027243957394', Distance_km=0.02567542243235302, MOTIF_REMORQUAGE="Constat d'infraction", Date_Time='2015-12-31', Year='2015', Month='12', Day='31', Mean_Temp='-2.7', Total_Rain=None, Total_Precip='0.7', Total_Snow=None, Spd_of_Max_G

In [9]:
joined_data.agg({'Year': 'max'}).collect()[0]

Row(max(Year)='2015')